In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [73]:
import torch.nn.functional as F
from torch import nn
import copy
import torch
import math
from torch.autograd import Variable

class Generator(nn.Module):
    "Define standard linear + softmax generation step."
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, d_model, layer, N, dropout=0.1):
        super(Encoder, self).__init__()
        self.pe = PositionalEncoding(d_model, dropout)
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x):
        x = self.pe(x)
        for layer in self.layers:
            x = layer(x)
        return self.norm(x)

class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x))
        return self.sublayer[1](x, self.feed_forward)

class Decoder(nn.Module):
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, memory):
        for layer in self.layers:
            x = layer(x, memory)
        return self.norm(x)

class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
 
    def forward(self, x, memory):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m))
        return self.sublayer[2](x, self.feed_forward)

def attention(query, key, value, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    key_dims = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(key_dims)
    product_attention = F.softmax(scores, dim = -1)
    if dropout is not None:
        product_attention = dropout(product_attention)
    return torch.matmul(product_attention, value), product_attention

class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)
    
def initialize(module):
    for p in module.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

def make_model(src_vocab, per_layer_embeddings, N=2, 
               d_model=128, d_ff=512, h=2, dropout=0.1):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
#     position = PositionalEncoding(d_model, dropout)

    enc = Encoder(d_model, EncoderLayer(d_model, c(attn), c(ff), dropout), N, dropout)
    
    decoders = []
    for num_embs in per_layer_embeddings:
        decoders.append(nn.Sequential(*[
            Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N)
            , Generator(d_model, num_embs)
        ]))
#     dec = Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N)
#     gen = Generator(d_model, tgt_vocab)
#     components = [enc, dec, src_emb, tgt_emb, gen]
        
    for dec in decoders:
        initialize(dec)
        
    initialize(enc)
    
    return enc, decoders

In [111]:
def attention(query, key, value, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    key_dims = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(key_dims)
    product_attention = F.softmax(scores, dim = -1)
    if dropout is not None:
        product_attention = dropout(product_attention)
    return torch.matmul(product_attention, value), product_attention

class MultiHeadedAttention(nn.Module):
    def __init__(self, number_of_heads, hidden_dimensions, dropout=0.1):
        super(MultiHeadedAttention, self).__init__()
        num_heads = number_of_heads
        h_dims = hidden_dimensions
        
        assert h_dims % num_heads == 0
        
        self.num_heads = num_heads
        self.h_dims = h_dims
        
        self.key_dims = h_dims // num_heads
        
        self.linears = nn.ModuleList([
            nn.Linear(h_dims, h_dims) for _ in range(num_heads)
        ])
        
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key=None, value=None):
        if key is None:
            key = value = query
        batch_size = query.shape[0]
        
        for lin in self.linears:
            query, key, value, = lin(query), lin(key), lin(value)
            
        x, self.attn = attention(query, key, value, dropout=self.dropout)
        
        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.h_dims)
        last_lin = self.linears[-1](x)

        return last_lin

In [107]:
from ipdb import set_trace
import numpy as np

options_in = 35
per_layer_options_out = [5, 4]
d_model = 64
batch_size = 128

embeddings = torch.rand(options_in, d_model)

batch_of_embeddings = [embeddings.clone().unsqueeze(0) for _ in range(batch_size)]
batch_of_embeddings = torch.cat(batch_of_embeddings)

batch_of_masks = torch.tensor(np.random.randint(options_in, size=(batch_size, 1))).float()

for emb, mask in zip(batch_of_embeddings, batch_of_masks):
    emb[mask.int().item():] = 0

In [143]:
-torch.log(torch.rand(50)*99 + 1)

tensor([-2.6822, -4.3477, -3.8235, -4.1139, -1.2362, -1.5059, -3.8071,
        -3.9914, -4.5322, -2.0083, -4.2997, -4.3520, -2.6503, -4.2345,
        -3.8087, -2.8087, -4.1410, -3.9539, -2.8093, -3.0644, -4.5267,
        -3.9771, -3.8214, -3.0253, -4.2359, -3.7245, -1.5530, -0.6964,
        -0.5198, -4.3731, -4.0754, -4.5581, -1.4649, -4.5653, -3.6201,
        -4.3613, -4.5328, -4.0076, -4.0592, -2.8047, -3.4677, -4.0840,
        -3.6456, -4.0577, -2.9157, -3.3373, -3.5771, -3.8372, -3.4502,
        -4.0155])

In [108]:
mha = MultiHeadedAttention(4, 64)

In [109]:
mha(batch_of_embeddings)

> <ipython-input-106-9cdffb7d617a>(35)forward()
     33         set_trace()
     34 
---> 35         return last_lin

ipdb> last_lin.shape
torch.Size([128, 35, 64])
ipdb> exit


BdbQuit: 

In [20]:
enc, decoders = make_model(options_in, per_layer_options_out, d_model=d_model)

In [22]:
encode_step = enc(batch_of_embeddings)

In [112]:
decoder = decoders[0]
decoder(encode_step, memory)